In [1]:
pos_data_path = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_pos/full"
neg_data_path = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_neg/full"

# read the datasets (huggingface)
# group by "ids"
# from neg dataset, remove responses w/ label=1
# merge the datasets: 50% of responses from each dataset.

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [2]:
# read the datasets (huggingface)
from datasets import load_from_disk
pos_data = load_from_disk(pos_data_path)
neg_data = load_from_disk(neg_data_path)

/lustre/fsw/portfolios/llmservice/users/sghotra/installs/miniforge3/envs/adv_verif/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
print(pos_data[0].keys())
print(len(pos_data))

dict_keys(['text', 'question', 'answer', 'id', 'target', 'response', 'label', 'gen_id'])
115361


In [ ]:
# from neg dataset, remove responses w/ label=1
print("Before filtering: ", len(neg_data))
neg_data = neg_data.filter(lambda x: x["label"] == 0)
print("After filtering: ", len(neg_data))

Before filtering:  100540
After filtering:  95331


In [5]:
# group by "ids"
pos_data_grp = {}
for i in range(len(pos_data)):
    id = pos_data[i]["id"]
    
    if id in pos_data_grp:
        pos_data_grp[id].append(pos_data[i])
    else:
        pos_data_grp[id] = [pos_data[i]]


neg_data_grp = {}
for i in range(len(neg_data)):
    
    id = neg_data[i]["id"]
    if id in neg_data_grp:
        neg_data_grp[id].append(neg_data[i])
    else:
        neg_data_grp[id] = [neg_data[i]]



In [11]:
# lens = [len(pos_data_grp[id]) for id in pos_data_grp.keys()]
# print(sum(lens))
# print(len(pos_data_grp))
# print(len(neg_data_grp[0]))

115361
7473
28


In [6]:
print(neg_data_grp[0][0].keys())

dict_keys(['text', 'question', 'answer', 'id', 'target', 'response', 'label', 'gen_id'])


In [6]:
import random
random.seed(6730)

# merge the pos and neg datasets: 50% of responses from each dataset.

incomplete_dpoints = 0
incomplete_dpoints_neg = 0
incomplete_dpoints_pos = 0

merged_data = []
all_ids = set(pos_data_grp.keys()) | set(neg_data_grp.keys())
for id in all_ids:
    pos_dp_h = []
    neg_dp_h = []

    if id in neg_data_grp:
        neg_dp = neg_data_grp[id]
        # print(neg_dp)
        # print(len(neg_dp))
        random.shuffle(neg_dp)
        neg_dp_h = neg_dp[:8]

        for dp in neg_dp_h:
            dp["policy"] = "neg"
    
    
    if id in pos_data_grp:
        pos_dp = pos_data_grp[id]
        random.shuffle(pos_dp)
        # FIXME: remove it
        n = max(8, 16 - len(neg_dp_h))
        pos_dp_h = pos_dp[:n]

        for dp in pos_dp_h:
            dp["policy"] = "pos"
    

    # if len(neg_dp_h) < 8:
    #     neg_dp_h = neg_dp.shuffle()[:8]
    #     if len(neg_dp_h) < 8:
    #         pos_dp_h = pos_dp.shuffle()[:8]
    
    # if len(pos_dp_h) < 8:
    #     pos_dp_h = pos_dp.shuffle()[:8]

    dp = pos_dp_h + neg_dp_h
    merged_data.extend(dp)

    if len(dp) < 16:
        incomplete_dpoints += 1
    
print(f"Number of incomplete datapoints: {incomplete_dpoints}")

Number of incomplete datapoints: 49


In [8]:
print(len(all_ids))
print("Merged data: ", len(merged_data))
print(merged_data[0].keys())

7473


In [8]:
# print a random some sample
rand_i = list(range(len(merged_data)))
random.shuffle(rand_i)
rand_i = rand_i[:5]

for i in rand_i:
    print(merged_data[i]['policy'])
    print(merged_data[i]['label'])
    print()
    print(merged_data[i]['text'])
    print("===================================")


pos
0

In his phone gallery, Malachi has the number of selfies he took last year and this year in the ratio of 10:17. If he has 2430 photos in his gallery, how many more photos did he take this year than last year? Step 1: There are 120 pages in the book. Julie reads 12 pages yesterday and 12 x 2 pages today. So today she reads 12 x 2 = 24 pages.
Step 2: Then there are 120 - 12 - 24 = 84 pages left. If she wants to read half, she must read 84 ÷ 2 = 42 pages.
The final answer is 42

neg
0

Linda owns a store. She sells jeans at 11 dollars each and tees at 8 dollars each. If she sold 7 tees and 4 jeans in a day, how much money, in dollars, did she have at the end of the day? Solution:
Step 1: Yesterday Julie read 12 pages and she read the same amount in total on the remaining days from today onwards, so she read 12 + (12 + 12 + 12) = 48 pages today and in total for the remaining days.
Step 2: Total pages read is 12 (yesterday) + 48 = 60.
Step 3: She started with 120 pages, so she has 120

In [10]:
# random select 500 ids for val set
val_ids = random.sample(all_ids, 500)

train_data = []
val_data = []

for dp in merged_data:
    if dp["id"] in val_ids:
        val_data.append(dp)
    else:
        train_data.append(dp)

print("Train data: ", len(train_data))
print("Val data: ", len(val_data))

# Now, split val data based on policy into pos_val and neg_val
pos_val_data = []
neg_val_data = []

for dp in val_data:
    if dp["policy"] == "pos":
        pos_val_data.append(dp)
    else:
        neg_val_data.append(dp)

print("Pos val data: ", len(pos_val_data))
print("Neg val data: ", len(neg_val_data))

# save the datasets
from datasets import Dataset
from datasets import DatasetDict


train_data_dict = {key: [d[key] for d in train_data] for key in train_data[0]}
train_dataset = Dataset.from_dict(train_data_dict)
# train_dataset.save_to_disk(output_dir)


# output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/val_pos"

val_pos_data_dict = {key: [d[key] for d in pos_val_data] for key in pos_val_data[0]}
val_pos_dataset = Dataset.from_dict(val_pos_data_dict)
# val_dataset.save_to_disk(output_dir)


# output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/val_neg"

val_neg_data_dict = {key: [d[key] for d in neg_val_data] for key in neg_val_data[0]}
val_neg_dataset = Dataset.from_dict(val_neg_data_dict)
# val_dataset.save_to_disk(output_dir)

output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged"
data = DatasetDict({"train": train_dataset, "val_pos": val_pos_dataset, "val_neg": val_neg_dataset})

data.save_to_disk(output_dir)



/tmp/ipykernel_3066778/2016821634.py:2: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  val_ids = random.sample(all_ids, 500)


Train data:  111436
Val data:  7976
Pos val data:  4036
Neg val data:  3940


Saving the dataset (1/1 shards): 100%|██████████| 3940/3940 [00:00<00:00, 423667.07 examples/s]


In [2]:
train_output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/train"
val_pos_output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/val_pos"
val_neg_output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/val_neg"

output_dir = "/lustre/fsw/portfolios/llmservice/users/sghotra/data/gen/pv_methd/preproc_gsm8k_merged/tmp"

import datasets
from datasets import DatasetDict

train_dataset = datasets.load_from_disk(train_output_dir)
val_pos_dataset = datasets.load_from_disk(val_pos_output_dir)
val_neg_dataset = datasets.load_from_disk(val_neg_output_dir)

data = DatasetDict({"train": train_dataset, "val_pos": val_pos_dataset, "val_neg": val_neg_dataset})

data.save_to_disk(output_dir)




Saving the dataset (0/1 shards):   0%|          | 0/107055 [00:00<?, ? examples/s]

Saving the dataset (1/1 shards): 100%|██████████| 3722/3722 [00:00<00:00, 147626.43 examples/s]


In [ ]:
data = datasets.load_from_disk(output_dir)